In [1]:
import random
import sympy
from sympy.ntheory.modular import solve_congruence

In [2]:
# pick a family of small primes
prime_set = [3,5,7,11,13,17]
k = len(prime_set)

assert all(sympy.isprime(prime) is True for prime in prime_set) is True

In [3]:
# divide the set in half and find products of primes
u = 1; v = 1

for i, prime in enumerate(prime_set):
    if i < len(prime_set)/2:
        u = u * prime
    else:
        v = v * prime

In [4]:
# product of all primes
sigma = u*v

In [5]:
while True:
    # pick large prime numbers
    a = sympy.randprime(100, 200)
    b = sympy.randprime(100, 200)

    assert sympy.isprime(a)
    assert sympy.isprime(b)

    # calculate two primes from chosen ones
    p = (2*a*u)+1
    q = (2*b*v)+1

    if sympy.isprime(p) and sympy.isprime(q):
        break

In [6]:
n = p*q
phi = (p-1)*(q-1)
print(f"{n=}, {phi=}")

n=30618288353, phi=30617326740


In [7]:
# choose a generator. its order must be phi(n)/4?
g = 131
assert g < n

In [8]:
print(f"public keys: {sigma}, {n}, {g}")
print(f"private keys: {p}, {q}")

public keys: 255255, 30618288353, 131
private keys: 32971, 928643


# encryption

In [9]:
m = 202
assert m < sigma

def encrypt(m):
    return pow(g, m, n)

c = encrypt(m)
print(f"ciphertext is {c}")

ciphertext is 11140580205


# decryption

In [10]:
def decrypt(c):
    remainders = []
    for i, prime in enumerate(prime_set):
        ci = pow(c, int(phi/prime), n)

        for j in range(0, prime):
            if ci == pow(g, int((j*phi)/prime), n):
                # print(f"m_{i} = {j}")
                remainders.append(j)
    
    congruences = []
    for i in range(0, k):
        print(f"m mod {prime_set[i]} = {remainders[i]}")
        congruences.append((remainders[i], prime_set[i]))
    
    # chinese remainder problem
    ms = solve_congruence(*congruences)
    ms = [i for i in ms if i < sigma]
    m = ms[0]
    print(f"Then m = {m}")
    return m


In [11]:
decrypt(c)

m mod 3 = 1
m mod 5 = 2
m mod 7 = 6
m mod 11 = 4
m mod 13 = 7
m mod 17 = 15
Then m = 202


202

# homomorphic features

In [12]:
m1 = 100
m2 = 200

c1 = encrypt(m1)
c2 = encrypt(m2)

In [13]:
(c1*c2) % n == encrypt(m1+m2)

True

In [14]:
decrypt((c1*c2) % n) == m1+m2

m mod 3 = 0
m mod 5 = 0
m mod 7 = 6
m mod 11 = 3
m mod 13 = 1
m mod 17 = 11
Then m = 300


True